In [2]:
#!pip install numpy==1.22
#!pip install pandas==1.3.4
!pip install keras==2.12.0
!pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-manylinux1_x86_64.whl (295.5 MB)
  Using cached tbb-2021.9.0-py2.py3-none-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

2023-05-01 18:12:35.171493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.650791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.653209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 18:12:38.447472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
#Split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors.head()
#predictors.shape
#target.shape

n_cols = predictors.shape[1] # number of predictors
n_cols


8

In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [8]:
from keras import Sequential
from tensorflow.keras import layers
from keras.layers import Dense

In [9]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(5, activation='relu'))
    #model.add(Dense(5, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

1)Randomly split the data into a training and test sets by holding 30% of the data for testing. 

In [10]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.30,random_state=42)

#predictors_train.shape
target_test.shape
np.array(target_test).reshape(103, 3)#use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


array([[52.91, 55.9 , 74.5 ],
       [35.3 , 10.54, 44.28],
       [23.69, 45.37, 37.4 ],
       [48.85, 25.1 , 10.73],
       [40.2 , 49.8 , 33.01],
       [21.92, 40.06, 19.35],
       [37.17, 33.09, 31.38],
       [37.96, 48.72,  6.9 ],
       [38.6 , 30.85, 11.17],
       [41.05, 52.96, 13.22],
       [52.3 , 39.6 , 51.86],
       [66.42, 14.94, 33.02],
       [31.64, 39.42, 13.66],
       [51.96, 15.58,  6.28],
       [38.07, 50.24, 10.03],
       [63.4 , 55.55, 36.15],
       [21.75,  4.83, 57.21],
       [47.78, 29.41, 12.37],
       [64.02, 34.4 , 25.37],
       [13.71, 31.72, 19.77],
       [39.42, 13.52, 35.34],
       [46.8 , 33.  , 18.03],
       [33.4 , 12.18, 27.66],
       [24.43, 11.65, 24.05],
       [ 9.31, 25.1 , 26.14],
       [11.58, 55.26, 60.95],
       [61.86, 14.59, 44.21],
       [34.9 , 41.68, 39.59],
       [41.05, 35.3 , 44.61],
       [33.72, 21.5 , 20.08],
       [31.65, 75.5 , 15.04],
       [56.61, 53.52, 50.66],
       [31.35, 45.71, 19.11],
       [32

## Train and Test the Network


In [11]:
#Build the model

model = regression_model()


2)  Train the model on the training data using 50 epochs

In [12]:
# train the model with fit
model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)

Epoch 1/50
16/16 - 2s - loss: 65204.1875 - val_loss: 45830.2539 - 2s/epoch - 119ms/step
Epoch 2/50
16/16 - 0s - loss: 34895.5234 - val_loss: 24031.3633 - 180ms/epoch - 11ms/step
Epoch 3/50
16/16 - 0s - loss: 18545.0430 - val_loss: 12580.1729 - 201ms/epoch - 13ms/step
Epoch 4/50
16/16 - 0s - loss: 9717.3340 - val_loss: 6403.0981 - 154ms/epoch - 10ms/step
Epoch 5/50
16/16 - 0s - loss: 5043.3696 - val_loss: 3313.8538 - 151ms/epoch - 9ms/step
Epoch 6/50
16/16 - 0s - loss: 2779.5671 - val_loss: 1911.8989 - 130ms/epoch - 8ms/step
Epoch 7/50
16/16 - 0s - loss: 1798.6123 - val_loss: 1351.4053 - 141ms/epoch - 9ms/step
Epoch 8/50
16/16 - 0s - loss: 1408.9252 - val_loss: 1154.6920 - 168ms/epoch - 10ms/step
Epoch 9/50
16/16 - 0s - loss: 1272.0813 - val_loss: 1085.8246 - 159ms/epoch - 10ms/step
Epoch 10/50
16/16 - 0s - loss: 1216.3572 - val_loss: 1053.7700 - 200ms/epoch - 13ms/step
Epoch 11/50
16/16 - 0s - loss: 1182.8315 - val_loss: 1030.1166 - 138ms/epoch - 9ms/step
Epoch 12/50
16/16 - 0s - loss:

In [13]:
#Predictions

predictions = model.predict(predictors_test)
np.array(predictions).reshape(103, 3) #use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


10/10 [==============================] - 0s 3ms/step


array([[62.7889  , 47.92364 , 53.629467],
       [47.940975, 37.52879 , 44.39921 ],
       [35.445736, 34.15814 , 31.331202],
       [35.557323, 30.88244 , 21.566324],
       [66.18215 , 48.02496 , 30.582178],
       [39.349087, 42.407284, 30.841745],
       [40.436184, 32.07597 , 31.95463 ],
       [25.17074 , 38.216778, 29.261454],
       [31.178812, 35.374157, 16.36033 ],
       [35.577297, 32.673256, 30.520548],
       [26.0021  , 35.825558, 33.81605 ],
       [28.54046 , 32.196102, 35.80534 ],
       [30.402948, 41.40118 , 29.33407 ],
       [34.002666, 22.941614, 28.655802],
       [27.742716, 48.87849 , 27.619928],
       [49.046505, 51.29449 , 54.4471  ],
       [28.67782 , 26.725855, 44.701992],
       [30.055582, 41.210598, 30.278788],
       [40.427715, 49.275875, 31.391779],
       [17.252985, 24.66868 , 17.233347],
       [41.16775 , 34.394283, 27.200235],
       [52.118664, 27.983423, 25.751474],
       [43.92445 , 24.786554, 32.6768  ],
       [29.76002 , 32.007717, 26.6

3) Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.


In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
metrics = mean_squared_error(target_test, predictions, multioutput = 'raw_values')
metrics

array([202.20112544])

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [16]:
metrics1 = np.zeros(50) #Incializando array
for i in range(49):
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)
    predictions = model.predict(predictors_test)
    metrics1[i] = np.array(mean_squared_error(target_test, predictions, multioutput = 'raw_values'))
    print(metrics1[i])

Epoch 1/50
16/16 - 0s - loss: 210.9964 - val_loss: 167.5777 - 200ms/epoch - 12ms/step
Epoch 2/50
16/16 - 0s - loss: 208.2647 - val_loss: 167.1428 - 109ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 204.9394 - val_loss: 165.9805 - 102ms/epoch - 6ms/step
Epoch 4/50
16/16 - 0s - loss: 203.1230 - val_loss: 163.9496 - 107ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 199.3027 - val_loss: 162.6109 - 105ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 196.4427 - val_loss: 161.4387 - 106ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 195.1819 - val_loss: 160.4113 - 105ms/epoch - 7ms/step
Epoch 8/50
16/16 - 0s - loss: 193.6018 - val_loss: 159.1879 - 104ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 189.0798 - val_loss: 158.2275 - 110ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 189.1646 - val_loss: 156.4287 - 104ms/epoch - 6ms/step
Epoch 11/50
16/16 - 0s - loss: 184.3178 - val_loss: 154.9380 - 104ms/epoch - 7ms/step
Epoch 12/50
16/16 - 0s - loss: 183.0300 - val_loss: 152.9582 -

In [22]:
#array with 
metrics1

array([133.80110862, 120.11016713, 114.76422453, 112.21116114,
       112.03436721, 110.32741865, 108.59521923, 107.60473073,
       114.08660268, 108.46620808, 121.03055167, 107.25636117,
       118.42233131, 106.93051301,  99.3081116 ,  80.07584474,
        71.54889699,  67.41793725,  61.23777085,  55.61185354,
        55.47638318,  55.84936563,  50.4236515 ,  50.48633757,
        56.27094008,  50.71335546,  48.82732692,  49.40091151,
        48.53604288,  50.84979852,  55.21693376,  50.7180406 ,
        50.67965689,  49.16336618,  48.17245245,  49.88015313,
        49.90976562,  48.13244302,  48.15847027,  48.89170182,
        50.09892361,  47.70316926,  51.96178889,  47.98698899,
        58.75844685,  48.50077836,  47.76884076,  51.72112591,
        49.42414472,   0.        ])

5. Report the mean and the standard deviation of the mean squared errors.

In [23]:
#The mean value

from statistics import mean

metrics1_avg = mean(metrics1)
metrics1_std = np.std(metrics1)

print("The mean of the MSN array is: ", metrics1_avg, " and the standard deviation is: ", metrics1_std)


The mean of the MSN array is:  70.01045368976844  and the standard deviation is:  30.00135545596085
